In [1]:
path = "/Users/mcdicjh2/Desktop/gecco22/"
from sklearn.metrics.cluster import adjusted_rand_score

# Recursive function to support translation of MST into initial GA solutions

def recursive_links(node_index, node_origin, x, temp_edges, binary_links, a, mst):
    

   # print(node_index,node_origin)
    
    # Draw link 
    if x[node_index] != -1:
        #print("I have already been here - going backwards")
        return

    # Set link to originating node - # Decode corresponding to full adjacency matrix
    for j in range(0,len(a[node_index])):        
        if a[node_index][j] == node_origin:
            x[node_index] = j+1
            binary_links[node_index] = node_origin
            temp_edges.append([node_index,node_origin])
                
    # Start recursion on all neighbors in MST
    for i in range(0,len(mst[node_index])):
        recursive_links(mst[node_index][i],node_index, x, temp_edges, binary_links, a, mst)
    
    return

In [2]:
# Set up EA in pymoo

import numpy as np
from igraph import *
import igraph
import pandas as pd
from pymoo.util.misc import stack
from pymoo.model.problem import Problem

from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.factory import get_termination

termination = get_termination("n_gen", 1000)
popsize = 50  


# PyMOO Problem Definition for the GA
class MyProblem(Problem):
    
    def __init__(self,g1,g2,g3,a,vertices,xl,d,binary_links,n_var,lower):
        self.g1=g1
        self.g2=g2
        self.g3=g3
        self.a=a
        self.vertices=vertices
        self.xl=xl
        self.xu=d
        self.binary_links=binary_links
        self.n_var=n_var
        self.lower=lower
        super().__init__(n_var=n_var,
                         n_obj=3,
                         n_constr=0,
                         # Set upper bound for each decision variable using information about node degrees
                         xl=xl,                         
                         xu=d,
                         elementwise_evaluation=True)


    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 0
        f2 = 0
        f3 = 0
        ctr0 = 0
        ctr1 = 0
        
        sol_edges = []
        # Allow self loops in encoding which are interpreted as "no edges"
        for i in range(0,self.n_var):
            if x[i] > 0:
                sol_edges.append([i,self.a[i][x[i]-1]])

         
        # Construct bipartite graph
        t = Graph.Bipartite(self.vertices,sol_edges)
        
        # Decoding: Identify unconnected components to define communities
        c = t.clusters()
        m = c.membership
        k = max(c.membership)+1
        
        # Evaluate both projections with respect to those communities
        m1 = self.g1.modularity(m[0:self.lower],weights=self.g1.es['weight'])#
        m2 = self.g2.modularity(m[self.lower:self.n_var],weights=self.g2.es['weight'])#
        m3 = self.g3.modularity(m)
        
        
        out["F"] = [-m1, -m2, k]


  





In [3]:
# Now actually run the GA
from pymoo.optimize import minimize
import multiprocessing
import time


    
def worker():
    print("Starting")
    name = multiprocessing.current_process().name
    
 
    # Read in graph and associated data
    g_org = Graph.Read_Edgelist(path+"Graph"+name+".dat")
    truth = pd.read_csv(path+"Graph"+name+".truth.dat", sep=',',header=None)
    groundtruth=truth.iloc[:, 1].values.tolist()
    v = pd.read_csv(path+"Graph"+name+".vertices.dat", sep=',',header=None)
    vertices=v.iloc[:, 1].values.tolist()
    edges = g_org.get_edgelist()
 
    # Construct bipartite instance
    g3 = Graph.Bipartite(vertices, edges)
    n_var = len(vertices)
    lower = vertices.count(0)
    

    # Determine both projections
    g1, g2 = g3.bipartite_projection(multiplicity=True)

    # Determine adjacency matrix
    a = g3.get_adjlist()


    # Determine number of possible edges for mutation
    d = g3.indegree()

    xl = np.empty(n_var)
    xl.fill(0)

    

    
    
    # Generate Minimum Spanning Tree

    t = g3.spanning_tree()

    e = t.get_edgelist()

    adj = t.get_adjlist()

    x = np.empty(n_var)
    x.fill(-1)
    x = x.tolist()

    binary_links = np.empty(n_var)
    binary_links.fill(-1)
    binary_links = binary_links.tolist()
    
    
    elementwise_problem = MyProblem(g1,g2,g3,a,vertices,xl,d,binary_links,n_var,lower)
    mp_result = pd.DataFrame(columns=['k','m', 'm1', 'm2', 'ari'])

    for it in range (0,1):

       # Minimum Spanning Tree Initialization        
        mst = t.get_adjlist()

        temp_edges = []
        for i in range(0,n_var):
            if x[i] == -1:
                x[i] = 0
                for j in range(0,len(mst[i])):
                    recursive_links(mst[i][j],i, x, temp_edges, binary_links, a, mst)
           
            
        pop = np.tile(x, (popsize, 1))
        c= g3.clusters()
        ctr = 0 

        # Initialization: Fast Greedy
        for i in range(len(c),1+min(50,popsize,n_var)):

            # Use different greedy solutions for diversity
            test = g3.community_fastgreedy()
            test = test.as_clustering(i)#i

            test=test.membership
            for j in range(0,n_var):
                if pop[ctr][j] != 0 and test[a[j][pop[ctr][j]-1]] != test[j]:
                    pop[ctr][j] = 0

            ctr = ctr +1

        # Initialization: Random Solutions
        for i in range(ctr,popsize):
            maxk = random.randint(0,20)
            for k in range(0,maxk):
                index = random.randint(0,n_var-1)
                pop[i][index] = 0


        algorithm = NSGA2(
            pop_size=popsize,
            n_offsprings=popsize,
            sampling=pop,
            crossover=get_crossover("int_ux", prob=0.3),
            mutation=get_mutation("int_pm"),
            eliminate_duplicates=True
        )



        res = minimize(elementwise_problem,
                   algorithm,
                   termination,
                   seed=1,
                   save_history=True,
                   verbose=True)

        # Collate results and eliminate duplicates

        for n in range(0,len(res.X)):
            sol_edges = []
            for i in range(0,n_var):
                if res.X[n][i] > 0:
                    sol_edges.append([i,a[i][res.X[n][i]-1]])
            t = Graph.Bipartite(vertices,sol_edges)
            c= t.clusters()

            m = c.membership
            m3 = g3.modularity(m)
            ari = adjusted_rand_score(groundtruth,m)
            m1 = g1.modularity(m[0:lower],weights=g1.es['weight'])#
            m2 = g2.modularity(m[lower:n_var],weights=g2.es['weight'])


            mp_result.loc[it*30+n] = len(c), m3, m1, m2, ari

    mp_result.drop_duplicates(keep="first",inplace=True)
    mp_result.to_csv(path+"Graph"+name+".mp.csv")



# Parallelization over potentially multiple runs (change 1 below e.g. to 30)
for run in range(0,30):
    worker1 = multiprocessing.Process(name=str(run), target=worker)
    worker1.start()
    

Starting
StartingStarting

Starting
Starting
Starting
Starting
Starting
Starting
StartingStarting

Starting
Starting==============================================================================================================

n_gen |  n_eval |  n_nds  |     eps      |  indicator  n_gen |  n_eval |  n_nds  |     eps      |  indicator  
n_gen |  n_eval |  n_nds  |     eps      |  indicator  Startingn_gen |  n_eval |  n_nds  |     eps      |  indicator  



=======================================================Starting=======================================================


n_gen |  n_eval |  n_nds  |     eps      |  indicator      1 |      30 |       4 |            - |            -    1 |      30 |       2 |            - |            -
    1 |      32 |       2 |            - |            -


Starting

    1 |      31 |       2 |            - |            -
n_gen |  n_eval |  n_nds  |     eps      |  indicator  =======================================================


StartingStartin